In [5]:
import sys

import torch
import numpy as np

from attacks import FGSMContainer
from basemodels import TorchModelContainer, IrisNN
from datasets import DATASET_LIST, DataContainer, get_dataset_list
# from defences import DefenceContainer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
print(sys.version)
print(*sys.path, sep='\n')

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
/home/lukec/.vscode/extensions/ms-python.python-2020.2.64397/pythonFiles
/home/lukec/.vscode/extensions/ms-python.python-2020.2.64397/pythonFiles/lib/python
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/venv/lib/python3.6/site-packages
/home/lukec/Downloads/jax/build
/home/lukec/.local/lib/python3.6/site-packages
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/home/lukec/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython


In [7]:
print('Avaliable datasets:')
print(get_dataset_list())

Avaliable datasets:
['MNIST', 'CIFAR10', 'SVHN', 'BankNote', 'BreastCancerWisconsin', 'HTRU2', 'Iris', 'WheatSeed']


In [8]:
DATA_ROOT = 'data'
BATCH_SIZE = 64

# image datasets: {'MNIST', 'CIFAR10', 'SVHN'}
# quantitative datasets: {'BankNote', 'BreastCancerWisconsin', 'HTRU2', 'Iris', 'WheatSeed'}
NAME = 'Iris'
print(f'Starting {NAME} data container...')
print(DATASET_LIST[NAME])

dc = DataContainer(DATASET_LIST[NAME], DATA_ROOT)
dc(size_train=0.8, normalize=True)

num_features = dc.dim_data[0]
num_classes = dc.num_classes
print('Features:', num_features)
print('Classes:', num_classes)

## model in {BCNN, IrisNN, MnistCnnCW}
model = IrisNN(num_features, num_classes, hidden_nodes=16)  # for Iris
model_name = model.__class__.__name__
print('Using model:', model_name)

mc = TorchModelContainer(model, dc)
mc.fit(epochs=50, batch_size=BATCH_SIZE)

Starting Iris data container...
{'name': 'Iris', 'type': 'quantitative', 'size': 150, 'num_classes': 3, 'dim_data': (4,)}
Loading data...
Preparing DataFrame...
Reading from data/iris.data
Spliting train/test sets into numpy arrays...
Successfully load data! Time taken:  0m 0.0s
Features: 4
Classes: 3
Using model: IrisNN
Using device: cuda:0
{'lr': 0.01, 'momentum': 0.9}
[ 1/50]  0m 0.0s - Train Loss: 1.0582 Acc: 57.5000% - Test Loss: 1.1009 Acc: 43.3333%
[ 2/50]  0m 0.0s - Train Loss: 1.0561 Acc: 63.3333% - Test Loss: 1.0972 Acc: 53.3333%
[ 3/50]  0m 0.0s - Train Loss: 1.0525 Acc: 66.6667% - Test Loss: 1.0923 Acc: 56.6667%
[ 4/50]  0m 0.0s - Train Loss: 1.0474 Acc: 69.1667% - Test Loss: 1.0855 Acc: 56.6667%
[ 5/50]  0m 0.0s - Train Loss: 1.0409 Acc: 69.1667% - Test Loss: 1.0777 Acc: 56.6667%
[ 6/50]  0m 0.0s - Train Loss: 1.0334 Acc: 69.1667% - Test Loss: 1.0695 Acc: 56.6667%
[ 7/50]  0m 0.0s - Train Loss: 1.0252 Acc: 69.1667% - Test Loss: 1.0612 Acc: 56.6667%
[ 8/50]  0m 0.0s - Train

In [9]:
attack = FGSMContainer(mc)

In [10]:
x1 = np.random.randn(2, 4).astype(np.float32)
x2 = np.random.randn(2, 4).astype(np.float32)
print(attack.get_l2_norm(x1, x2))
print(attack.get_l2_norm(x1, x1))
print(attack.get_l2_norm(x1, x1-1e-3))

[3.0524302 0.674838 ]
[0. 0.]
[0.00200009 0.00199997]


In [11]:
attack.generate()

array([[0.33333328, 0.20833333, 0.5084746 , 0.49999997],
       [0.66666657, 0.20833333, 0.81355935, 0.7083333 ],
       [0.5277777 , 0.3333333 , 0.6440678 , 0.7083333 ],
       [0.5277777 , 0.5833334 , 0.7457627 , 0.9166666 ],
       [0.5833333 , 0.5       , 0.7288136 , 0.9166666 ],
       [0.49999994, 0.3333333 , 0.6271186 , 0.45833334],
       [0.66666657, 0.41666666, 0.71186435, 0.9166666 ],
       [0.02777775, 0.37500003, 0.0677966 , 0.04166667],
       [0.02777775, 0.41666666, 0.05084745, 0.04166667],
       [0.4166667 , 0.2916667 , 0.5254237 , 0.375     ]], dtype=float32)